# SEMA-3D

The MIT License (MIT)

Copyright (c) 2016 AYLIEN

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import os
import copy
import math
import json
import pandas as pd
import numpy as np
import pickle
import scipy

from pathlib import Path

import esm
from esm.data import BatchConverter
from esm.inverse_folding.util import CoordBatchConverter

import torch
from torch.utils.data import Dataset
from torch import nn
from tqdm import tqdm

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction


from biotite.structure.residues import get_residues
import matplotlib.pyplot as plt
import seaborn as sns


import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import r2_score, mean_squared_error, auc, plot_precision_recall_curve


In [ ]:
def esmStructDataset(pdb_path,chain):
    structure = esm.inverse_folding.util.load_structure(str(pdb_path), chain.upper())
    resi_index = get_residues(structure)[0]
    resi_aa    = get_residues(structure)[1]
    resi_keys     = []
    cn =[]
    binary = []
    for resi_index_,resi_aa_ in zip(get_residues(structure)[0],get_residues(structure)[1]):
        key = (str(resi_aa_),resi_index_)
        cn.append(None)
        binary.append(None)
        resi_keys.append(key)    
    coords, seq = esm.inverse_folding.util.extract_coords_from_structure(structure)#structure)
    return {"pdb_id":pdb_path,"seq":seq,"chain":chain,"coords":coords,"cn":cn,"binary":binary,"residues":resi_keys}

class epitopes_Dataset(Dataset):
    def __init__(self, epitope_data):
        self.epitope_data    = epitope_data
        #_, alphabet          = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
        self.batch_converter = CoordBatchConverter(alphabet)

    def __getitem__(self, idx):
        item = {}
        coords = self.epitope_data[idx]["coords"]
        seq    = self.epitope_data[idx]["seq"]     
        batch = [(coords, None, seq)]
        coords, confidence, strs, tokens, padding_mask = self.batch_converter(batch)
        item['coords']           = coords
        item['confidence']       = confidence
        item['tokens']           = tokens
        item['padding_mask']     = padding_mask
        item['labels_cn']        = torch.unsqueeze(torch.Tensor(self.epitope_data[idx]["cn"]),     0).to(torch.float32)
        item['labels_binary']    = torch.unsqueeze(torch.LongTensor(self.epitope_data[idx]["binary"]), 0)#.to(torch.float32)
        #item['labels_binary_finite']    = torch.unsqueeze(torch.LongTensor(self.epitope_data[idx]["binary_finite"]), 0)#.to(torch.float32)

        return item

    def __len__(self):
        return len(self.epitope_data)
  

In [ ]:
class ESM1vForTokenClassification(nn.Module):
    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels    
        self.esm1v, self.esm1v_alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
        self.classifier = nn.Linear(512, self.num_labels)

    def forward(self, coords, padding_mask, confidence, tokens):
        prev_output_tokens = tokens[:, :-1]
        target = tokens[:, 1:]
        target_padding_mask = (target == alphabet.padding_idx)
        feat, x = self.esm1v.forward(coords, padding_mask, confidence, prev_output_tokens, features_only = True)
        f = feat[0,:,:]
        tt = torch.transpose(feat,1,2)
        logits = self.classifier(tt )
        return SequenceClassifierOutput(logits=logits)    

def loadModel(path = "./finetuned_models/mse_S_ens_0"):
    model = ESM1vForTokenClassification().cuda()
    wts   = pickle.load(open(path,'rb'))
    model.load_state_dict(wts)
    return model

In [ ]:
sema_model = loadModel()
_, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()

sema_models = []
for i in range(5):
    sema_models.append(loadModel(f"./finetuned_models/mse_S_ens_{i}"))

In [ ]:
def run(PDB_PATH, chain):
    struct = esmStructDataset(PDB_path,chain)#temp_pdb/7E3O.pdb","R")
    custom_seq = struct["seq"]
    batch = [(struct["coords"], None, custom_seq)]#struct["seq"])] 
    batch_converter = CoordBatchConverter(alphabet)
    coords, confidence, strs, tokens, padding_mask = batch_converter(batch)
    with torch.no_grad():
        preds = []
        for sema_model in sema_models:
            pred = sema_model.forward(coords.cuda(),
                                     padding_mask.cuda(),
                                     confidence.cuda(),
                                     tokens.cuda())
            pred = pred.logits[:,:,1].squeeze().detach().cpu().numpy()
            preds.append(pred)
        pred = np.average(np.array(preds),axis=0)
    return pred



In [ ]:
sema_predictions = run("./test_protein.pdb","A")